The point of this assignment is to clean and wrangle data using Numpy and Pandas from a survey of Halloween candy to prepare it for a machine learning project. 

## Data Set

The data set that we will be using is the 2017 Halloween Candy Hierarchy data set as discussed in this [boingboing](https://boingboing.net/2017/10/30/the-2017-halloween-candy-hiera.html) article.  You can also read more about the data in the [Science Creative Quarterly](https://www.scq.ubc.ca/so-much-candy-data-seriously/).

The following are the rating instructions from the survey:  

> Basically, consider that feeling you get when you receive this item in your Halloween haul. Does it make you really happy (JOY)? Or is it something that you automatically place in the junk pile (DESPAIR)? MEH for indifference, and you can leave blank if you have no idea what the item is.

Note that the original data set has been slightly altered from its original state, and if you wanted to perform any analysis for future projects, you would need to download the data directly from the links above.


## Note

The end goal for this project is to clean the data so that we could then create a machine learning model. We want to see if we are able to predict a person's gender based purely on their candy preferences. Although, you will not be creating a model for this assignment, only cleaning the data. The results of the models that I used after cleaning the data are provided at the end of this notebook.

## Initial Import & Exploration

In [2]:
# initial imports
import pandas as pd
import numpy as np

# Do not change this option; This allows the CodeGrade auto grading to function correctly
pd.set_option('display.max_columns', 20)

I'll start by importing our data and creating a DataFrame called `candy`.  We need to include `encoding='iso-8859-1'` during the import because there are special characters in the data that Pandas doesn't recognize. This happens a lot when attempting to import data where the public is able to input answers, especially if there are foreign language characters included. The normal encoding for Pandas is `utf-8`, so changing the encoding allows Pandas to recognize those special characters. 


In [3]:
# read_csv with iso-8859-1 encoding
candy_full = pd.read_csv('candy.csv', encoding='iso-8859-1')

# copy to new DF so that we can have a copy of the original import if needed
candy = candy_full.copy()

Take a brief look at the data by using `head()`.

In [4]:
# first five rows
candy.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


View information about the DataFrame.

In [5]:
# check info about the DataFrame
candy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Columns: 120 entries, Internal ID to Click Coordinates (x, y)
dtypes: float64(4), int64(1), object(115)
memory usage: 2.3+ MB


I can make columns display by setting the `max_cols` argument equal to the number of columns in the data set.

In [6]:
# check info, set max_cols
candy.info(max_cols=120)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 120 columns):
 #    Column                                                                                 Non-Null Count  Dtype  
---   ------                                                                                 --------------  -----  
 0    Internal ID                                                                            2479 non-null   int64  
 1    Q1: GOING OUT?                                                                         2368 non-null   object 
 2    Q2: GENDER                                                                             2437 non-null   object 
 3    Q3: AGE                                                                                2394 non-null   object 
 4    Q4: COUNTRY                                                                            2414 non-null   object 
 5    Q5: STATE, PROVINCE, COUNTY, ETC                                   

I can use a simple `for` loop if only looking for the column names.

In [7]:
# print a list of column names
for col in candy.columns:
    print(col)

Internal ID
Q1: GOING OUT?
Q2: GENDER
Q3: AGE
Q4: COUNTRY
Q5: STATE, PROVINCE, COUNTY, ETC
Q6 | 100 Grand Bar
Q6 | Anonymous brown globs that come in black and orange wrappers	(a.k.a. Mary Janes)
Q6 | Any full-sized candy bar
Q6 | Black Jacks
Q6 | Bonkers (the candy)
Q6 | Bonkers (the board game)
Q6 | Bottle Caps
Q6 | Box'o'Raisins
Q6 | Broken glow stick
Q6 | Butterfinger
Q6 | Cadbury Creme Eggs
Q6 | Candy Corn
Q6 | Candy that is clearly just the stuff given out for free at restaurants
Q6 | Caramellos
Q6 | Cash, or other forms of legal tender
Q6 | Chardonnay
Q6 | Chick-o-Sticks (we donÕt know what that is)
Q6 | Chiclets
Q6 | Coffee Crisp
Q6 | Creepy Religious comics/Chick Tracts
Q6 | Dental paraphenalia
Q6 | Dots
Q6 | Dove Bars
Q6 | Fuzzy Peaches
Q6 | Generic Brand Acetaminophen
Q6 | Glow sticks
Q6 | Goo Goo Clusters
Q6 | Good N' Plenty
Q6 | Gum from baseball cards
Q6 | Gummy Bears straight up
Q6 | Hard Candy
Q6 | Healthy Fruit
Q6 | Heath Bar
Q6 | Hershey's Dark Chocolate
Q6 | HersheyÕ

## Data Cleaning

Rename the column names that include the `Õ` character and replace it with an apostrophe.  

In [8]:
candy.columns = candy.columns.str.replace("Õ","'")


Take a look at how many duplicated rows we have in this file, assuming a duplicate is a row that has all of the exact same values.

In [9]:
duplicated_rows = len(candy[candy.duplicated()])

Take a look at how many duplicates there are if we consider a duplicate is any row with the same `Internal ID` number as another.

In [10]:
InternalID_duplicates = len(candy[candy.duplicated('Internal ID')])

Here, I drop any duplicates from the `candy` DataFrame.  Duplicates are to be defined as any row with the same `Internal ID` as another.

In [11]:
candy = candy.drop_duplicates('Internal ID',keep='first')

Remove the following columns: `Internal ID`, `Q5: STATE, PROVINCE, COUNTY, ETC`, `Q7: JOY OTHER`, `Q8: DESPAIR OTHER`, `Q9: OTHER COMMENTS`, `Unnamed: 113`, `Click Coordinates (x, y)`.

In [12]:
candy = candy.drop(['Internal ID','Q5: STATE, PROVINCE, COUNTY, ETC','Q7: JOY OTHER','Q8: DESPAIR OTHER','Q9: OTHER COMMENTS','Unnamed: 113','Click Coordinates (x, y)'],axis=1)

Let's take a look at the `Q2: GENDER` column since this will be what we are trying to predict. Take a look at the value counts for this column.

In [13]:
candy.value_counts('Q2: GENDER')

Q2: GENDER
Male                  1466
Female                 839
I'd rather not say      83
Other                   30
Name: count, dtype: int64

How many missing values are in the `Q2: GENDER` column? 

In [14]:
gender_missing_values = candy['Q2: GENDER'].isnull().sum()
gender_missing_values

42

Remove all rows with a missing value in the `Q2: GENDER` column.

In [15]:
candy = candy.dropna(subset = ['Q2: GENDER'])

In order to prepare this data for binary classification, I will select only the rows that contain either `Male` or `Female` in the `Q2: GENDER` column.

In [16]:
candy = candy[(candy['Q2: GENDER'] == 'Male') | (candy['Q2: GENDER'] == 'Female')]

Now, I'm going to work on filling some of the missing data with Pandas.

How many missing values are in the `Q1: GOING OUT?` column?

In [17]:
goingout_missing_vals = candy['Q1: GOING OUT?'].isnull().sum()

For a future analysis question, we are interested in those that we know will *definitely* go out for Halloween.  Because of this, I will fill all missing values in the `Q1: GOING OUT?` column with a `No` value.

In [18]:
candy['Q1: GOING OUT?'].fillna('No',inplace = True)

Double check that all GOING OUT column values are either a Yes or No.

In [19]:
candy.value_counts(['Q1: GOING OUT?'])

Q1: GOING OUT?
No                2007
Yes                298
Name: count, dtype: int64

Next, I will fill any missing values in the `candy` DataFrame for those columns (going from `Q6 | 100 Grand Bar` to `Q11: DAY`) with the string `NO_ANSWER`. 

In [20]:
candy.loc[:,'Q6 | 100 Grand Bar':'Q11: DAY'] = candy.loc[:,'Q6 | 100 Grand Bar':'Q11: DAY'].fillna('NO_ANSWER')


For all four `Q12: Media` columns in the `candy` DataFrame, I will fill the missing values with `0.0`.

In [21]:
candy.loc[:,'Q12: MEDIA [Daily Dish]':'Q12: MEDIA [Yahoo]'] = candy.loc[:,'Q12: MEDIA [Daily Dish]':'Q12: MEDIA [Yahoo]'].fillna(value = 0.0)



Now, let's look at the very messy `Q4: COUNTRY` column and see what we can do about it. First, run the code below to look at the different unique values in the data.

In [22]:
# check unique values
candy['Q4: COUNTRY'].unique()

array(['USA ', 'USA', 'us', 'usa', nan, 'canada', 'Canada', 'Us', 'US',
       'Murica', 'United States', 'uk', 'United Kingdom', 'united states',
       'Usa', 'United States ', 'United staes',
       'United States of America', 'UAE', 'England', 'UK', 'canada ',
       'United states', 'u.s.a.', '35', 'france',
       'United States of America ', 'america', 'U.S.A.', 'finland',
       'unhinged states', 'Mexico', 'Canada ', 'united states of america',
       'US of A', 'The United States', 'North Carolina ', 'Unied States',
       'Netherlands', 'germany', 'Europe', 'U S', 'u.s.', 'U.K. ',
       'Costa Rica', 'The United States of America', 'unite states',
       'U.S.', '46', 'Australia', 'Greece', 'USA? Hard to tell anymore..',
       "'merica", '45', 'United State', '32', 'France', 'australia',
       'Can', 'Canae', 'Trumpistan', 'Ireland', 'United Sates', 'Korea',
       'California', 'Unites States', 'Japan', 'USa', 'South africa',
       'I pretend to be from Canada, but I am

In [23]:
# check the Q4: COUNTRY number of unique values
candy['Q4: COUNTRY'].nunique()

115

We want to clean up this data to only include four areas: USA, Canada, Europe (the continent, not necessarily the European Union), and Other.

- Fill in all missing values with `Other`
- Code Australia as `Other`
- Combine all USA entries together as `USA`
- Combine Canadian entries as `CA`
- Combine European entries as `EU`
- Everything else gets coded as `Other`

Fill the missing values in the `Q4: COUNTRY` column with `Other`.

In [24]:
candy['Q4: COUNTRY'].fillna('Other',inplace = True)


In [25]:
# check unique values 
candy['Q4: COUNTRY'].nunique()

116

In [26]:
candy['Q4: COUNTRY'].unique()

array(['USA ', 'USA', 'us', 'usa', 'Other', 'canada', 'Canada', 'Us',
       'US', 'Murica', 'United States', 'uk', 'United Kingdom',
       'united states', 'Usa', 'United States ', 'United staes',
       'United States of America', 'UAE', 'England', 'UK', 'canada ',
       'United states', 'u.s.a.', '35', 'france',
       'United States of America ', 'america', 'U.S.A.', 'finland',
       'unhinged states', 'Mexico', 'Canada ', 'united states of america',
       'US of A', 'The United States', 'North Carolina ', 'Unied States',
       'Netherlands', 'germany', 'Europe', 'U S', 'u.s.', 'U.K. ',
       'Costa Rica', 'The United States of America', 'unite states',
       'U.S.', '46', 'Australia', 'Greece', 'USA? Hard to tell anymore..',
       "'merica", '45', 'United State', '32', 'France', 'australia',
       'Can', 'Canae', 'Trumpistan', 'Ireland', 'United Sates', 'Korea',
       'California', 'Unites States', 'Japan', 'USa', 'South africa',
       'I pretend to be from Canada, but 

Combine all Australia entries into `Other`.  

In [27]:
#candy[candy['Q4: COUNTRY'].str.contains('australia',case=False)] = 'Other'
candy['Q4: COUNTRY'].replace(['Australia','australia'], 'Other', inplace=True)

In [28]:
# check number of unique values
candy['Q4: COUNTRY'].nunique()

114

Combine all United States entries together into `USA`.  These would include the following:


In [29]:
candy['Q4: COUNTRY'].replace(['USA ', 'USA', 'us', 'usa', 'Us', 'US', 'Murica', 'United States', 'united states', 'Usa', 
'United States ', 'United staes', 'United States of America', 'United states', 'u.s.a.', 'United States of America ', 'america',
'U.S.A.', 'unhinged states', 'united states of america', 'US of A', 'The United States', 'North Carolina ', 'Unied States', 'U S', 'u.s.', 
'The United States of America', 'unite states','U.S.', 'USA? Hard to tell anymore..', "'merica", 'United State', 'United Sates', 'California',
'Unites States', 'USa', 'I pretend to be from Canada, but I am really from the United States.', 'Usa ', 'United Stated', 'New Jersey', 
'United ststes', 'America', 'United Statss', 'murrika', 'USA! USA! USA!', 'USAA', 'united States ', 'N. America', 'USSA', 'U.S. ', 'u s a', 
'United Statea', 'united ststes', 'USA USA USA!!!!'], 'USA', inplace=True)


I will combine the Canadian entries (both upper and lower case) and label them as `CA`.

In [30]:
#candy['Q4: COUNTRY'].replace('canada', 'CA')
#candy['Q4: COUNTRY'].replace('Canada', 'CA')
#candy['Q4: COUNTRY'].replace('canada ', 'CA')
#candy['Q4: COUNTRY'].replace('Canada ', 'CA')
#candy['Q4: COUNTRY'].replace('Can', 'CA')
#candy['Q4: COUNTRY'].replace('Canae', 'CA')
#candy['Q4: COUNTRY'].replace('Canada`', 'CA')
#candy['Q4: COUNTRY'].replace('CANADA', 'CA')

candy['Q4: COUNTRY'].replace(['canada','Canada','canada ','Canada ','Can','Canae','Canada`','CANADA'], 'CA', inplace=True)

Next, I will combine the European entries and label them as `EU`.

In [31]:
candy['Q4: COUNTRY'].replace(['uk', 'United Kingdom', 'England', 'UK', 'france', 'finland', 'Netherlands', 'germany', 'Europe',
                   'U.K. ', 'Greece', 'France', 'Ireland', 'Uk', 'Germany', 'Scotland', 'UK ', 'Denmark', 'France ',
                   'Switzerland', 'Scotland ', 'The Netherlands', 'Ireland ', 'spain', 'Sweden', 'United kingdom'], 'EU', inplace=True)


Finally, I will combine the other entries and label them as `Other`.

In [32]:
#candy.loc[~candy['Q4: COUNTRY'].isin(['USA', 'CA', 'EU'],), 'Q4: COUNTRY'] = 'Other'
candy['Q4: COUNTRY'].replace(['UAE', '35', 'Mexico', 'Costa Rica',
       '46', '45', '32', 'Trumpistan', 'Korea', 'Unites States', 'Japan',
       'South africa', 'UD', 'South Korea', 'Indonesia', 'endland',
       'soviet canuckistan', 'Singapore', 'China', 'Taiwan', 'hong kong',
       'Hong Kong', 'Narnia', "I don't know anymore", 'Fear and Loathing','subscribe to dm4uz3 on youtube'], 'Other', inplace=True)

Double check that I only have four unique values in the `Q4: COUNTRY` column: `USA`, `Other`, `CA`, and `EU`

In [33]:
# check values
candy['Q4: COUNTRY'].nunique()

4

We now want to look at the `Q3: AGE` column. Let's look at all the unique values.

In [34]:
# check unique age values
candy['Q3: AGE'].unique()

array(['44', '49', '40', '23', nan, '53', '33', '43', '56', '64', '37',
       '48', '54', '36', '45', '25', '34', '35', '38', '58', '50', '47',
       '16', '52', '63', '65', '41', '27', '31', '59', '61', '46', '42',
       '62', '29', '39', '32', '28', '69', '67', '30', '22', '51', '70',
       '24', '19', 'Old enough', '57', '60', '66', '12', 'Many', '55',
       '72', '?', '21', '11', 'no', '9', '68', '20', '6', '10', '71',
       '13', '26', '45-55', '7', '39.4', '74', '18', 'older than dirt',
       '17', '15', '8', '75', '5u', 'Enough', 'Over 50', '90', '76',
       'sixty-nine', 'ancient', '77', 'OLD', 'old', '73', '70 1/2', '14',
       'MY NAME JEFF', '4', '59 on the day after Halloween', 'old enough',
       'your mom', 'I can remember when Java was a cool new language',
       '60+'], dtype=object)

I will put these values into the following categorical bins: `unknown`, `17 and under`, `18-25`, `26-35`, `36-45`, `46-55`, and `56+`.  Missing values should be replaced with the `unknown` category

First, we will replace any non-numeric value (those with text as mentioned above) with a missing value.  This will allow me to turn the other values into floats so that I can bin them.

In [35]:
# create True/False index
age_index = candy['Q3: AGE'].str.isnumeric()

# for the index, fill missing values with False
age_index = age_index.fillna(False)

# select Age column for only those False values from index and code as missing
candy.loc[~age_index, 'Q3: AGE'] = np.nan



In [36]:

#change data type to float
candy['Q3: AGE'] = candy['Q3: AGE'].astype(float)

#create bins
candy['Q3: AGE'] = pd.cut(x=candy['Q3: AGE'], bins = [-1,17,25,35,45,55,100],
                                            labels = ['17 and under','18-25','26-35','36-45','46-55','56+'])

# add category 'unknown'
candy['Q3: AGE'] = candy['Q3: AGE'].cat.add_categories('unknown')

#reorder categories
candy['Q3: AGE'] = candy['Q3: AGE'].cat.reorder_categories(['unknown','17 and under','18-25','26-35','36-45','46-55','56+'])

# code missing values as 'unknown'
candy['Q3: AGE'] = candy['Q3: AGE'].fillna('unknown')

Double check the categories

In [37]:
# double check categories
candy['Q3: AGE'].cat.categories

Index(['unknown', '17 and under', '18-25', '26-35', '36-45', '46-55', '56+'], dtype='object')

In [38]:
# look at the value counts of each category
candy['Q3: AGE'].value_counts()

Q3: AGE
36-45           768
46-55           525
26-35           520
56+             298
18-25            85
unknown          60
17 and under     49
Name: count, dtype: int64

Make sure the output is 0 for null values.

In [39]:
candy.isnull().sum()

Q1: GOING OUT?             0
Q2: GENDER                 0
Q3: AGE                    0
Q4: COUNTRY                0
Q6 | 100 Grand Bar         0
                          ..
Q11: DAY                   0
Q12: MEDIA [Daily Dish]    0
Q12: MEDIA [Science]       0
Q12: MEDIA [ESPN]          0
Q12: MEDIA [Yahoo]         0
Length: 113, dtype: int64

Reset the index for `candy` ensuring that it goes from 0 to n-1.  

In [40]:
candy = candy.reset_index(drop=True)

## Feature Engineering 

Feature engineering is the process of transforming raw data into features that better represent the underlying problem to the predictive models.  In this section, I will create a new column called "net_feelies" (calculated by the authors as the total joy count minus the total despair count).

First, let's narrow down the data to make working with it easier.

Select only the Q6 candy columns (`Q6 | 100 Grand Bar` through `Q6 | York Peppermint Patties`) in the data set and save this as a new DataFrame called `candy_reduced`.

In [41]:
candy_reduced = candy.loc[:,'Q6 | 100 Grand Bar':'Q6 | York Peppermint Patties']

Next, we will create two Series, one with JOY counts and one with DESPAIR counts to add to the `candy_reduced` data.

Next, I will create a Series called `joy_count` that lists total counts for JOY for each column, making sure to keep it in the same order as the columns in the `candy_reduced` DataFrame.

In [42]:
joy_count = candy_reduced[candy_reduced == 'JOY'].count()


I will do the same as above except I will create a Series called `despair_count` that lists the total counts for DESPAIR for each column.

In [43]:
despair_count = candy_reduced[candy_reduced == 'DESPAIR'].count()

Take the transpose of the `candy_reduced` DataFrame and save this transposed data as `candy_reduced_transpose`.

In [44]:
candy_reduced_transpose = np.transpose(candy_reduced)

I will add a new column called "joy_count" using the `joy_count` Series above and a new column called 'despair_count" using the `despair_count` Series above to the `candy_reduced_transpose` DataFrame.

In [45]:
candy_reduced_transpose['joy_count'] = joy_count
candy_reduced_transpose['despair_count'] = despair_count

Adding a new column to the `candy_reduced_transpose` DataFrame called "net_feelies" that takes the `joy_count` column and subtracts the `despair_count` column.

In [46]:
candy_reduced_transpose['net_feelies'] = (joy_count-despair_count)

I will select only the `joy_count`, `despair_count`, and `net_feelies` columns from the `candy_reduced_transpose` DataFrame, and sort this DataFrame in descending order by `net_feelies` and save as `candy_net_sorted`.

In [47]:
candy_net_sorted = candy_reduced_transpose.loc[:,'joy_count':'net_feelies'].sort_values('net_feelies',ascending=False)

## Encoding

I now want to get the `candy` DataFrame ready to run a machine learning algorthim to determine if we could predict a person's gender based on what candy they prefer.

Make a copy of the `candy` DataFrame and save this new DataFrame as `candy_encode`.

In [48]:
candy_encode = candy.copy()

For the `candy_encode` DataFrame, I will replace any `Female` values with `0` and any `Male` values with `1`.

In [49]:
candy_encode = candy_encode.replace('Female','0')
candy_encode = candy_encode.replace('Male','1')


Selecting only the `Q2: GENDER` column from `candy_encode` and save this as `candy_response` as a Series.
Drop the following columns from the `candy_encode` DataFrame: `Q2: GENDER`,`Q1: GOING OUT?`,`Q3: AGE`,`Q4: COUNTRY`,`Q10: DRESS`,`Q11: DAY`, `Q12: MEDIA [Daily Dish]`,`Q12: MEDIA [Science]`,`Q12: MEDIA [ESPN]`,`Q12: MEDIA [Yahoo]`.  I will save the remaining columns as `candy_features`.

In [50]:
candy_response = (candy_encode['Q2: GENDER']).astype(int)
candy_features = candy_encode.drop(['Q2: GENDER','Q1: GOING OUT?','Q3: AGE','Q4: COUNTRY','Q10: DRESS',
'Q11: DAY','Q12: MEDIA [Daily Dish]','Q12: MEDIA [Science]','Q12: MEDIA [ESPN]','Q12: MEDIA [Yahoo]'],axis=1)
candy_response = candy_response.reset_index(drop=True)
candy_features = candy_features.reset_index(drop=True)

Using Panda's `get_dummies()` to encode the `candy_features` data, making sure to set `drop_first=True`. Saved as `candy_features_encoded`.

In [51]:
candy_features_encoded = pd.get_dummies(candy_features,drop_first=True)

**Code Check:** Make sure that the `candy`, `candy_features`, `candy_response` and `candy_features_encoded` have an index that goes from 0 to n-1 or your final CodeGrade tests will not pass.

## Final Analysis

I have now cleaned your data and prepared it to be passed to a machine learning model.  

My instructor for this course then created models using Random Forest, Logistic Regression, and XGBoost algorithms, and they all returned around 70% accuracy rates. However, the other accuracy metrics didn't look as good. Given the metrics that were calculated, I would say that based only on this data, using candy preference is not that great of an indicator of someone's gender.

